## LOAD LIBRARIES

In [6]:
import os
import pandas as pd
import re
import json
import openpyxl

import requests
import time

## ІФС (Індекс фінансового стресу) | FSI (Financial Stress Index)

In [13]:
r = requests.get("https://bank.gov.ua/ua/stability/fsi#:~:text=%D0%86%D0%BD%D0%B4%D0%B5%D0%BA%D1%81%20%D1%84%D1%96%D0%BD%D0%B0%D0%BD%D1%81%D0%BE%D0%B2%D0%BE%D0%B3%D0%BE%20%D1%81%D1%82%D1%80%D0%B5%D1%81%D1%83%20(%D0%86%D0%A4%D0%A1)%20%E2%80%93,%D0%BD%D0%B5%20%D0%B2%D0%BA%D0%B0%D0%B7%D1%83%D1%94%20%D0%BD%D0%B0%20%D0%BC%D0%B0%D0%B9%D0%B1%D1%83%D1%82%D0%BD%D1%96%20%D1%80%D0%B8%D0%B7%D0%B8%D0%BA%D0%B8.")
fsi_pd = pd.DataFrame(json.loads(re.search(r'JSON\.parse\(\'(.*?)\'\)', r.text).group(1)))
fsi_pd.columns = ['date', 'fsi']
fsi_pd.to_csv('./raw data/fsi_index.csv', index=False)

https://bank.gov.ua/ua/open-data/api-dev

## 6. Довідник державних цінних паперів

## 7. Кредити, надані депозитними корпораціями

In [ ]:
# generate range of dates for api process

start_date = pd.Timestamp('20160901')
today = pd.Timestamp.today()

month_diff = (today.year - start_date.year) * 12 + today.month - start_date.month
periods_list = pd.date_range(start='20160901', periods=month_diff, freq='MS').strftime('%Y%m%d').tolist()

In [ ]:
df_all = pd.DataFrame()
for m in periods_list:
    dr = requests.get(f'https://bank.gov.ua/NBUStatService/v1/statdirectory/loan?date={m}&json')
    dr = pd.DataFrame(dr.json())
    df_all = pd.concat([df_all, dr])
    print(m)
    time.sleep(3)
df_all.to_csv('./raw data/loans.csv', index=False)

In [ ]:
# donwload raw data

loans_downloaded = pd.read_csv('./raw data/loans.csv')

periods_list = list(set(periods_list) - set(loans_downloaded['dt'].astype(str).drop_duplicates()))

print(f"Absent periods - {len(periods_list)}")

if len(periods_list) >0:
    df_all = pd.DataFrame()
    for m in periods_list:
        dr = requests.get(f'https://bank.gov.ua/NBUStatService/v1/statdirectory/loan?date={m}&json')
        dr = pd.DataFrame(dr.json())
        df_all = pd.concat([df_all, dr])
        print(m)
        time.sleep(3)
    df_all = pd.concat([df_all, loans_downloaded])
    df_all.to_csv('./raw data/loans.csv', index=False)

In [ ]:
# donwload aggregated data

pd.pivot_table(loans_downloaded, index='dt', columns='txt', values='value', aggfunc='sum').reset_index().to_csv('./agg data/loans_agg.csv')

## 8. Депозити, залучені депозитними корпораціями

In [ ]:
# generate range of dates for api process

start_date = pd.Timestamp('20160901')
today = pd.Timestamp.today()

month_diff = (today.year - start_date.year) * 12 + today.month - start_date.month
periods_list = pd.date_range(start='20160901', periods=month_diff, freq='MS').strftime('%Y%m%d').tolist()

In [ ]:
df_all = pd.DataFrame()
for m in periods_list:
    dr = requests.get(f'https://bank.gov.ua/NBUStatService/v1/statdirectory/deposit?date={m}&json')
    dr = pd.DataFrame(dr.json())
    df_all = pd.concat([df_all, dr])
    print(m)
    time.sleep(3)
df_all.to_csv('./raw data/deposits.csv', index=False)

In [ ]:
# donwload raw data

deposits_downloaded = pd.read_csv('./raw data/deposits.csv')

periods_list = list(set(periods_list) - set(deposits_downloaded['dt'].astype(str).drop_duplicates()))

print(f"Absent periods - {len(periods_list)}")

if len(periods_list) >0:
    df_all = pd.DataFrame()
    for m in periods_list:
        dr = requests.get(f'https://bank.gov.ua/NBUStatService/v1/statdirectory/deposit?date={m}&json')
        dr = pd.DataFrame(dr.json())
        df_all = pd.concat([df_all, dr])
        print(m)
        time.sleep(3)
    df_all = pd.concat([df_all, deposits_downloaded])
    df_all.to_csv('./raw data/deposits.csv', index=False)

In [ ]:
# donwload aggregated data

pd.pivot_table(deposits_downloaded, index='dt', columns='txt', values='value', aggfunc='sum').reset_index().to_csv('./agg data/deposits_agg.csv')

## 9. Цінні папери, акції та інші форми участі в капіталі в портфелі депозитних корпорацій (крім Національного банку України), та цінні папери, крім акцій, випущені депозитними корпораціями (крім Національного банку України)

In [ ]:
# generate range of dates for api process

start_date = pd.Timestamp('20160901')
today = pd.Timestamp.today()

month_diff = (today.year - start_date.year) * 12 + today.month - start_date.month
periods_list = pd.date_range(start='20160901', periods=month_diff, freq='MS').strftime('%Y%m%d').tolist()

In [ ]:
df_all = pd.DataFrame()
for m in periods_list:
    dr = requests.get(f'https://bank.gov.ua/NBUStatService/v1/statdirectory/securities?date={m}&json')
    dr = pd.DataFrame(dr.json())
    df_all = pd.concat([df_all, dr])
    print(m)
    time.sleep(3)
df_all.to_csv('./raw data/secr.csv', index=False)

In [ ]:
# donwload raw data

secr_downloaded = pd.read_csv('./raw data/secr.csv')

periods_list = list(set(periods_list) - set(secr_downloaded['dt'].astype(str).drop_duplicates()))

print(f"Absent periods - {len(periods_list)}")

if len(periods_list) >0:
    df_all = pd.DataFrame()
    for m in periods_list:
        dr = requests.get(f'https://bank.gov.ua/NBUStatService/v1/statdirectory/securities?date={m}&json')
        dr = pd.DataFrame(dr.json())
        df_all = pd.concat([df_all, dr])
        print(m)
        time.sleep(3)
    df_all = pd.concat([df_all, secr_downloaded])
    df_all.to_csv('./raw data/secr.csv', index=False)

In [ ]:
# donwload aggregated data

pd.pivot_table(secr_downloaded, index='dt', columns='txt', values='value', aggfunc='sum').reset_index().to_csv('./agg data/secr_agg.csv')

## 10. Процентні ставки депозитних корпорацій (крім Національного банку України) за новими кредитами та новими депозитами

In [28]:
# generate range of dates for api process

start_date = pd.Timestamp('20160901')
today = pd.Timestamp.today()

month_diff = (today.year - start_date.year) * 12 + today.month - start_date.month
periods_list = pd.date_range(start='20160901', periods=month_diff, freq='MS').strftime('%Y%m%d').tolist()

In [29]:
df_all = pd.DataFrame()
for m in periods_list:
    dr = requests.get(f'https://bank.gov.ua/NBUStatService/v1/statdirectory/mir?date={m}&json')
    dr = pd.DataFrame(dr.json())
    df_all = pd.concat([df_all, dr])
    print(m)
    time.sleep(3)
df_all.to_csv('./raw data/dep_rates.csv', index=False)

20160901
20161001
20161101
20161201
20170101
20170201
20170301
20170401
20170501
20170601
20170701
20170801
20170901
20171001
20171101
20171201
20180101
20180201
20180301
20180401
20180501
20180601
20180701
20180801
20180901
20181001
20181101
20181201
20190101
20190201
20190301
20190401
20190501
20190601
20190701
20190801
20190901
20191001
20191101
20191201
20200101
20200201
20200301
20200401
20200501
20200601
20200701
20200801
20200901
20201001
20201101
20201201
20210101
20210201
20210301
20210401
20210501
20210601
20210701
20210801
20210901
20211001


In [ ]:
# donwload raw data

dep_rates_downloaded = pd.read_csv('./raw data/dep_rates.csv')

periods_list = list(set(periods_list) - set(dep_rates_downloaded['dt'].astype(str).drop_duplicates()))

print(f"Absent periods - {len(periods_list)}")

if len(periods_list) >0:
    df_all = pd.DataFrame()
    for m in periods_list:
        dr = requests.get(f'https://bank.gov.ua/NBUStatService/v1/statdirectory/mir?date={m}&json')
        dr = pd.DataFrame(dr.json())
        df_all = pd.concat([df_all, dr])
        print(m)
        time.sleep(3)
    df_all = pd.concat([df_all, dep_rates_downloaded])
    df_all.to_csv('./raw data/dep_rates.csv', index=False)

In [ ]:
# donwload aggregated data

pd.pivot_table(dep_rates_downloaded, index='dt', columns='txt', values='value', aggfunc='sum').reset_index().to_csv('./agg data/dept_rates_agg.csv')

## 11. Доходи та витрати банків

In [23]:
# generate range of dates for api process

start_date = pd.Timestamp('20090201')
today = pd.Timestamp.today()

month_diff = (today.year - start_date.year) * 12 + today.month - start_date.month
periods_list = pd.date_range(start='20090201', periods=month_diff, freq='MS').strftime('%Y%m%d').tolist()

In [25]:
# donwload raw data

rev_downloaded = pd.read_csv('./raw data/banks_rev.csv')

periods_list = list(set(periods_list) - set(rev_downloaded['dt'].astype(str).drop_duplicates()))

print(f"Absent periods - {len(periods_list)}")

if len(periods_list) >0:
    df_all = pd.DataFrame()
    for m in periods_list:
        rev = requests.get(f'https://bank.gov.ua/NBUStatService/v1/statdirectory/banksincexp?date={m}&period=m&json')
        df_rev = pd.DataFrame(rev.json())
        df_all = pd.concat([df_all, df_rev])
        print(m)
        time.sleep(3)
    df_all = pd.concat([df_all, rev_downloaded])
    df_all.to_csv('./raw data/banks_rev.csv', index=False)

Absent periods - 0


In [27]:
# donwload aggregated data

pd.pivot_table(rev_downloaded, index='dt', columns='txt', values='value', aggfunc='sum').reset_index().to_csv('./agg data/banks_rev_agg.csv')

## 12. Основні показники діяльності банків України

In [14]:
# generate range of dates for api process

start_date = pd.Timestamp('20160101')
today = pd.Timestamp.today()

month_diff = (today.year - start_date.year) * 12 + today.month - start_date.month
periods_list = pd.date_range(start='20160101', periods=month_diff, freq='MS').strftime('%Y%m%d').tolist()

In [21]:
# donwload raw data

mm_downloaded = pd.read_csv('./raw data/main_metrics.csv')

periods_list = list(set(periods_list) - set(mm_downloaded['dt'].astype(str).drop_duplicates()))

print(f"Absent periods - {len(periods_list)}")

if len(periods_list) >0:
    df_all = pd.DataFrame()
    for m in periods_list:
        mm = requests.get(f'https://bank.gov.ua/NBUStatService/v1/statdirectory/basindbank?date={m}&period=m&json')
        df_mm = pd.DataFrame(mm.json())
        df_all = pd.concat([df_all, df_mm])
        print(m)
        time.sleep(3)
    df_all = pd.concat([df_all, mm_downloaded])
    df_all.to_csv('./raw data/main_metrics.csv', index=False)

Absent periods - 0


In [22]:
# donwload aggregated data

pd.pivot_table(mm_downloaded, index='dt', columns='txt', values='value', aggfunc='sum').reset_index().to_csv('./agg data/main_metrics_agg.csv')

## 13. Дані фінансової звітності/згруповані балансові залишки банків України

In [2]:
# generate range of dates for api process

start_date = pd.Timestamp('20180201')
today = pd.Timestamp.today()

month_diff = (today.year - start_date.year) * 12 + today.month - start_date.month
periods_list = pd.date_range(start='20180201', periods=month_diff, freq='MS').strftime('%Y%m%d').tolist()

In [3]:
# donwload raw data

balances_downloaded = pd.read_csv('./raw data/balances.csv')

periods_list = list(set(periods_list) - set(balances_downloaded['dt'].astype(str).drop_duplicates()))

print(f"Absent periods - {len(periods_list)}")

if len(periods_list) >0:
    df_all = pd.DataFrame()
    for m in periods_list:
        balances = requests.get(f'https://bank.gov.ua/NBUStatService/v1/statdirectory/banksfinrep?date={m}&period=m&json')
        df_balances = pd.DataFrame(balances.json())
        df_all = pd.concat([df_all, df_balances])
        print(m)
        time.sleep(3)
    df_all = pd.concat([df_all, balances_downloaded])
    df_all.to_csv('./raw data/balances.csv', index=False)

Absent periods - 0


In [4]:
# donwload aggregated data

pd.pivot_table(balances_downloaded, index='dt', columns='txt', values='value', aggfunc='sum').reset_index().to_csv('./agg data/balances_agg.csv')